In [1]:
# Testing gauges and derived gauges

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from opencensus.metrics.export.gauge import LongGauge
from opencensus.metrics.label_key import LabelKey


lg = LongGauge("name", "description", "unit",
               [LabelKey("lk1", "first label key"), LabelKey("lk2", "second label key")])
lg

LongGauge(descriptor.name="name", points=OrderedDict())

In [4]:
# no points, null metric
from datetime import datetime as dt

lg.get_metric(dt.now()) is None

True

In [5]:
# we get a different point for each set of label values
lp_en = lg.get_or_create_time_series(["one", "two"])
lp_es = lg.get_or_create_time_series(["uno", "dos"])

lp_en, lp_es

(GaugePointLong(0), GaugePointLong(0))

In [6]:
# check that we created default (0-value) points and can now get a metric
lg.get_metric(dt.now())

Metric((lk1="one", lk2="two", [0]), (lk1="uno", lk2="dos", [0]), descriptor.name="name")

In [7]:
# points are cached on the gauge, one per set of label values
assert lp_en == lg.get_or_create_time_series(["one", "two"])

In [8]:
# check that we can't add a double val to a long gauge
lp_en.add(10.0)

ValueError: GaugePointLong only supports integer types

In [9]:
lp_en.add(10)
lp_en, lp_es

(GaugePointLong(10), GaugePointLong(0))

In [10]:
lp_en.add(10)
lp_es.set(1)
lp_es.set(2)
lp_es.set(-1)
lp_en, lp_es

(GaugePointLong(20), GaugePointLong(-1))

In [11]:
met = lg.get_metric(dt.now())
met

Metric((lk1="one", lk2="two", [20]), (lk1="uno", lk2="dos", [-1]), descriptor.name="name")

In [12]:
met.time_series[0]

TimeSeries([20], label_values=('one', 'two'), start_timestamp=2019-02-14 10:27:12.696080)

In [13]:
met.time_series[0].points[0].value

ValueLong(20)

In [14]:
from opencensus.metrics.export.gauge import DoubleGauge

dg = DoubleGauge("name", "description", "unit",
                 [LabelKey("lk1", "first label key"), LabelKey("lk2", "second label key")])
dg

DoubleGauge(descriptor.name="name", points=OrderedDict())

In [15]:
dp1 = dg.get_or_create_time_series(["yi", "er"])
dp1.add(2.5)
dp1

GaugePointDouble(2.5)

In [16]:
# check that we get the same point
dp2 = dg.get_or_create_time_series(["yi", "er"])
dp2.add(2.5)
assert dp1 == dp2
dp2

GaugePointDouble(5.0)

In [17]:
default_point = lg.get_or_create_default_time_series()
default_point

GaugePointLong(0)

In [18]:
default_point.add(100)

In [19]:
lg.get_or_create_default_time_series().value

100

In [20]:
# check that the default ts shows up
lg.get_metric(dt.now())

Metric((lk1="one", lk2="two", [20]), (lk1="uno", lk2="dos", [-1]), (lk1="None", lk2="None", [100]), descriptor.name="name")

In [21]:
# and that it doesn't show up after we remove it
lg.remove_default_time_series()
lg.get_metric(dt.now())

Metric((lk1="one", lk2="two", [20]), (lk1="uno", lk2="dos", [-1]), descriptor.name="name")

In [22]:
# and that metrics work if there's only the default
defg = LongGauge("name", "description", "unit", [LabelKey("lk", "only label key")])
defg.get_or_create_default_time_series()
defg.get_metric(dt.now())

Metric((lk="None", [0]), descriptor.name="name")

In [23]:
from datetime import datetime as dt
import math

# make some floats
def wobbler():
    return round(math.cos(dt.now().microsecond), 3)

[wobbler() for ii in range(10)]

[0.827, 0.515, -0.994, 0.312, 0.734, -0.923, -0.822, 0.86, 0.107, -0.949]

In [24]:
# make some ints
def fibber(nums=[0, 1]):
    nums[:] = [nums[1], sum(nums)]
    return nums[1]    

[fibber() for ii in range(10)]

[1, 2, 3, 5, 8, 13, 21, 34, 55, 89]

In [25]:
fibber.im_func

AttributeError: 'function' object has no attribute 'im_func'

In [26]:
from opencensus.metrics.export.gauge import GaugePointLong
from opencensus.metrics.export.gauge import GaugePointDouble
from opencensus.metrics.export.gauge import DerivedGaugePoint

dgpl = DerivedGaugePoint(fibber, GaugePointLong())
dgpd = DerivedGaugePoint(wobbler, GaugePointDouble())

(dgpl.get_value(), dgpd.get_value())

(144, -0.15)

In [27]:
class FibHolder(object):
    def __init__(self):
        self.nums = [0, 1]
        
    def get_num(self):
        self.nums = [self.nums[1], sum(self.nums)]
        return self.nums[1]
    
fh = FibHolder()
fh.get_num.__self__

In [28]:
import weakref
from functools import wraps

# this snippet comes from the o'reilly python cookbook
# https://www.oreilly.com/library/view/python-cookbook/0596001673/ch05s15.html
class _weak_callable:
    def __init__(self, obj, func):
        self.__self__ = obj
        self.__func__ = func

    def __call__(self, *args, **kws):
        if self.__self__ is None:
            return self.__func__(*args, **kws)
        else:
            return self.__func__(self.__self__, *args, **kws)

class WeakMethod:
    def __init__(self, fn):
        try:
            self._obj = weakref.ref(fn.__self__)
            self._meth = fn.__func__
        except AttributeError:
            # It's not a bound method
            self._obj = None
            self._meth = fn
            
    def _getobj(self):
        if self._obj is None:
            return None
        return self._obj()
    
    def _dead(self):
        return self._obj is not None and self._obj() is None

    def __call__(self):
        if self._dead():
            return None
        return _weak_callable(self._getobj(), self._meth)
    
# and this is my own version, using 3-compatible __self__/__func__
# see
#    https://docs.python.org/2/reference/datamodel.html
class WeakrefWrapper(object):
    def __init__(self, obj, func):
        self.__self__ = weakref.ref(obj)
        self.__func__ = func

    def __call__(self):
        if self.__self__() is None:
            return None
        
        @wraps(self.__func__)
        def wrapped_func(*args, **kws):
            return self.__func__(self.__self__(), *args, **kws)
        return wrapped_func
        
def get_weakref(func):
    try:
        return WeakrefWrapper(func.__self__, func.__func__)
    except AttributeError:
        return weakref.ref(func)

In [29]:
wr = weakref.ref(fibber)
print(wr)
print(wr())
print(wr()())
wr()()

<weakref at 0x109b5d098; to 'function' at 0x109b4cd90 (fibber)>
<function fibber at 0x109b4cd90>
233


377

In [30]:
fake_func = get_weakref(fibber)
print(fake_func)
print(fake_func())
print(fake_func()())
fake_func()()

<weakref at 0x109b5d098; to 'function' at 0x109b4cd90 (fibber)>
<function fibber at 0x109b4cd90>
610


987

In [31]:
fh = FibHolder()
fake_method = get_weakref(fh.get_num)
print(fake_method)
print(fake_method())
print(fake_method()())
del fh
print(fake_method)
print(fake_method())
print(fake_method()())

<function FibHolder.get_num at 0x109b6b0d0>
1
None


TypeError: 'NoneType' object is not callable

In [32]:
fh = FibHolder()
weak_get_num = WeakMethod(fh.get_num)
print(weak_get_num())
print(weak_get_num()())
print(weak_get_num()())
print(weak_get_num()())
del fh
print(weak_get_num())
print(weak_get_num()())

1
2
3
None


TypeError: 'NoneType' object is not callable

In [33]:
fh.get_num()

NameError: name 'fh' is not defined

In [34]:
from opencensus.metrics.export.gauge import DerivedLongGauge
from opencensus.metrics.label_key import LabelKey


dlg = DerivedLongGauge("name", "description", "unit",
                       [LabelKey("lk1", "first label key"), LabelKey("lk2", "second label key")])
dlg

DerivedLongGauge(descriptor.name="name", points=OrderedDict())

In [35]:
fh = FibHolder()
point = dlg.create_default_time_series(fh.get_num)

In [36]:
dlg.points

OrderedDict([((None, None),
              DerivedGaugePoint(<bound method FibHolder.get_num of <__main__.FibHolder object at 0x109b84860>>))])

In [37]:
point.func.__class__

weakref.WeakMethod

In [38]:
point.func()

<bound method FibHolder.get_num of <__main__.FibHolder object at 0x109b84860>>

In [39]:
dlg.get_metric(dt.now())

Metric((lk1="None", lk2="None", [1]), descriptor.name="name")

In [40]:
dlg.points

OrderedDict([((None, None),
              DerivedGaugePoint(<bound method FibHolder.get_num of <__main__.FibHolder object at 0x109b84860>>))])